In [72]:
import scanpy as sc
from  os.path import join
import sys
import os
lib = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\classifying-response-to-immunotherapy\utilities\droplet_dataset'
lib2 = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\classifying-response-to-immunotherapy\utilities'
lib3 = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\classifying-response-to-immunotherapy\data_analysis'
lib4 = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\classifying-response-to-immunotherapy'
lib5 = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\classifying-response-to-immunotherapy\scripts'
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)
import os
from os.path import join
import sklearn
# from droplet_dataset import *
# from utilities import *
from matplotlib import pyplot
import numpy as np
import scipy
import pickle
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import random
from scipy.stats import pearsonr
from matplotlib.pyplot import figure
from termcolor import colored
import tables
from collections import Counter

In [73]:
PATH = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\Data\droplet_seq\DEBUG\1000_rand_cells.pkl'
cohort = pickle.load(open(PATH, 'rb'))

In [ ]:
def train_test_split(self, test_size=0.2, shuffle=True, stratify=True):
    """
    :param test_size: in proportion to the number of patients (not proportion to nubber of cells)
    :param shuffle:
    :param random_state:
    :param stratify: Always, equal ration of responder to non-responder between test-set and train-set.
    :return: x_train, x_test, and corresponding indexes
    """

    responders = list(set([p.patient_details for p in self.cells_information_list if p.response_label]))
    non_responders = list(set([p.patient_details for p in self.cells_information_list if not p.response_label]))
    if shuffle:
        random.shuffle(responders)
        random.shuffle(non_responders)

    train_responders = responders[:int(len(responders) * (1 - test_size))]
    test_responders = responders[int(len(responders) * (1 - test_size)):]
    train_non_responders = non_responders[:int(len(non_responders) * (1 - test_size))]
    test_non_responders = non_responders[int(len(non_responders) * (1 - test_size)):]

    train_patients = train_responders + train_non_responders
    test_patients = test_responders + test_non_responders

    train_idxs = [idx for idx, p in enumerate(self.cells_information_list['patient_details']) if p in train_patients]
    test_idxs = [idx for idx, p in enumerate(self.cells_information_list['patient_details']) if p in test_patients]

    x_train = self[train_idxs]
    x_test = self[test_idxs]

    return x_train, x_test, train_idxs, test_idxs

In [38]:
import random
res = list(set(cohort.samples))
print(res, end='\n\n')
random.shuffle(res)
print(res)

['M120', 'M103', 'M115', 'M129', 'M105', 'M122', 'M143', 'M106', 'M124', 'M126', 'M110', 'M118', 'M132', 'M136', 'M130', 'M145', 'M139', 'M134', 'M144', 'M116', 'M121', 'M127', 'M109', 'M100', 'M111', 'M97', 'M131', 'M138', 'M104', 'M112', 'M141', 'M133', 'M146', 'M123', 'M137', 'M108', 'M107', 'M140', 'M99', 'M101', 'M102', 'M135', 'M128', 'M98', 'M114']

['M126', 'M123', 'M127', 'M139', 'M100', 'M107', 'M145', 'M111', 'M98', 'M121', 'M112', 'M115', 'M136', 'M114', 'M105', 'M144', 'M129', 'M133', 'M104', 'M135', 'M141', 'M137', 'M128', 'M130', 'M124', 'M131', 'M132', 'M102', 'M97', 'M110', 'M120', 'M99', 'M146', 'M140', 'M122', 'M143', 'M109', 'M134', 'M108', 'M106', 'M103', 'M116', 'M101', 'M138', 'M118']


In [74]:
dd = [list(ii) for ii in list(dict(Counter(cohort.samples)).items())]
print(dd, end='\n\n')

random.shuffle(dd)
print(dd, end='\n\n')
ee = dd
for idx in range(1, len(ee)):
    ee[idx][1] += ee[idx-1][1]
print(ee, end='\n\n')

[['M100', 16], ['M101', 17], ['M102', 69], ['M103', 8], ['M104', 20], ['M105', 31], ['M106', 19], ['M107', 32], ['M108', 8], ['M109', 33], ['M110', 14], ['M111', 31], ['M112', 17], ['M114', 44], ['M115', 17], ['M116', 12], ['M118', 5], ['M120', 21], ['M121', 25], ['M122', 12], ['M123', 16], ['M124', 25], ['M126', 16], ['M127', 16], ['M128', 39], ['M129', 11], ['M130', 21], ['M131', 22], ['M132', 25], ['M133', 14], ['M134', 5], ['M135', 14], ['M136', 18], ['M137', 31], ['M138', 17], ['M139', 19], ['M140', 16], ['M141', 10], ['M143', 53], ['M144', 38], ['M145', 32], ['M146', 34], ['M97', 18], ['M98', 16], ['M99', 23]]

[['M99', 23], ['M124', 25], ['M112', 17], ['M122', 12], ['M115', 17], ['M128', 39], ['M104', 20], ['M108', 8], ['M103', 8], ['M140', 16], ['M109', 33], ['M130', 21], ['M141', 10], ['M136', 18], ['M144', 38], ['M138', 17], ['M145', 32], ['M120', 21], ['M123', 16], ['M100', 16], ['M111', 31], ['M114', 44], ['M107', 32], ['M101', 17], ['M126', 16], ['M146', 34], ['M143', 53],

In [71]:
test_set_size = cohort.number_of_cells * 0.2
number_of_cells_acc = np.array([ii[1] for ii in ee])
samples_acc = np.array([ii[0] for ii in ee])


# samples_acc[~(number_of_cells_acc < test_set_size)].tolist()

In [ ]:
def train_test(cohortq):
    import random
    res = list(set(cohort.samples))
    print(res, end='\n\n')
    random.shuffle(res)
    print(res)

In [40]:
# sorted(cohort.counts[0])[-100:]
# test_size = 0.2
# Counter(cohort.samples)

In [21]:
cohort.gene_names[1]

'TNMD'

In [20]:
cohort.samples[1]

'M100'